In [15]:
%pip install openai
%pip install langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain.llms import OpenAI

openai = OpenAI(temperature=0)

prompt="Create a Kubernetes deployment in Terraform. Provide common variables."
print(openai(prompt))



variable "name" {
  type = string
  description = "Name of the Kubernetes deployment"
}

variable "namespace" {
  type = string
  description = "Namespace of the Kubernetes deployment"
}

variable "image" {
  type = string
  description = "Image of the Kubernetes deployment"
}

variable "replicas" {
  type = number
  description = "Number of replicas of the Kubernetes deployment"
}

resource "kubernetes_deployment" "example" {
  metadata {
    name = var.name
    namespace = var.namespace
  }

  spec {
    replicas = var.replicas
    selector {
      match_labels = {
        app = var.name
      }
    }
    template {
      metadata {
        labels = {
          app = var.name
        }
      }
      spec {
        container {
          image = var.image
        }
      }
    }
  }
}


In [21]:
examples = [
    {
        "query": "Deploy an Nginx in Terraform with Seal webservice module.",
        "answer": """
        module "nginx" {{
            env          = {{}}
            image        = "nginx"
            limit_cpu    = ""
            limit_memory = ""
            image        = "nginx"
            ports        = [80]
            request_cpu  = "0.1"
            request_memory = "128Mi"
            source = "github.com/seal-io/modules//webservice/0.0.3?ref=release"
        }}
        """
    }, {
        "query": "Deploy an Nginx in Terraform with Seal webserevice module, and limit its cpu in 1.",
        "answer": """
        module "nginx" {{
            env          = {{}}
            image        = "nginx"
            limit_cpu    = "1"
            limit_memory = ""
            image        = "nginx"
            ports        = [80]
            request_cpu  = "0.1"
            request_memory = "128Mi"
            source = "github.com/seal-io/modules//webservice/0.0.3?ref=release"
        }}
        """
    }, {
        "query": "Deploy an Nginx in Terraform with Seal webserevice module, and add a new environment TEST=true for it.",
        "answer": """
        module "nginx" {{
            env = {{
                TEST = true
            }}
            image        = "nginx"
            limit_cpu    = "1"
            limit_memory = ""
            image        = "nginx"
            ports        = [80]
            request_cpu  = "0.1"
            request_memory = "128Mi"
            source = "github.com/seal-io/modules//webservice/0.0.3?ref=release"
        }}
        """
    }
]

In [23]:
from langchain import PromptTemplate, FewShotPromptTemplate

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
assistant.

Here are some examples:
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

print(openai(few_shot_prompt_template.format(query="Deploy an Nginx in Terraform with Seal webservice module, and open its 443 port too.")))


        module "nginx" {
            env          = {}
            image        = "nginx"
            limit_cpu    = ""
            limit_memory = ""
            image        = "nginx"
            ports        = [80, 443]
            request_cpu  = "0.1"
            request_memory = "128Mi"
            source = "github.com/seal-io/modules//webservice/0.0.3?ref=release"
        }
